# Метод Apriori ассоциативного анализа

Написанного Ю Мочизуки (Yu Mochizuki)
Устанавливаем

```
pip install apyori
copy apyori.py into your project.
python setup.py install
```


In [ ]:
import pandas as pd
# Данные https://www.kaggle.com/datasets/devchauhan1/market-basket-optimisationcsv
dataset = pd.read_csv('./data/Market_Basket_Optimisation.csv', header = None)
dataset.head()

### Очистка данных


In [ ]:
dataset.fillna(method = 'ffill',axis = 1, inplace = True)
dataset.head()

In [ ]:
from apyori import apriori

#создадим из них матрицу
transactions = []
for i in range(0, 7501): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])
""" 
и обучимся правилам. Обратите внимание, что пороговые значения мы выбираем сами в зависимости от того,
насколько "сильные" правила мы хотим получить
min_support -- минимальный support для правил (dtype = float).
min_confidence -- минимальное значение confidence для правил (dtype = float)
min_lift -- минимальный lift (dtype = float)
max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer) 
"""
result = list(apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 4, min_length = 2))
apriori

In [ ]:
import shutil, os 
from io import StringIO
from apyori import dump_as_json
from IPython.display import display, HTML
import json

output = []
for RelationRecord in result:
    o = StringIO()
    dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
df = pd.DataFrame(output)
df.sort_values(by=["support"], ascending=False)

### Результат

Видим сочетания:

- оливковое масло, макароны из цельнозерновой муки [olive oil, whole wheat pasta],
- говяжий фарш, зелень и перец, спагетти [ground beef, herb & pepper, spaghetti],
- эскалоп, макароны [escalope, pasta]
